In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


In [88]:
## Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
# data.head()

In [89]:
## Preprocess the data
### Drop irrelevant columns
columns_to_drop = ['RowNumber', 'CustomerId', 'Surname']
data = data.drop(columns=columns_to_drop, axis=1)
# data.head()

In [90]:
## Categorical variable - Gender and Geography
## Encode categorical variables
## Apply Label encoder on Gender
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [91]:
## Apply one hot encoding on Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
# geo_encoder.toarray()


In [ ]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

In [93]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
# geo_encoded_df

In [94]:
# Combine one hot encoded columns with the original data
data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df ], axis=1)
# data

In [95]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [96]:
## Divide the data set into independent and dependent features
X = data.drop(columns=['Exited'], axis=1)
y=data['Exited']

## Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.2, random_state=42)

## Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [97]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Implementation

In [98]:
## Import necessary librabries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime

In [ ]:
## Build the Sequential Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),## HL1 Connected with input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='sigmoid')
])

In [100]:
# model.summary()

In [101]:
import tensorflow
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [102]:
## Compile the model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [103]:
## Setup Tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir = "logs/fit/"+datetime.datetime.now().strftime('%Y%m%d-%h%M%S')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [108]:
## Setup EarlyStopping Callback
early_stopping_callback = EarlyStopping(monitor='var_loss', patience=5, mode='max',restore_best_weights=True)

In [ ]:
## Train the model with callbacks
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorboard_callback,early_stopping_callback ]
)

In [ ]:
## Save the model
model.save('model.h5')

In [111]:
## Load Tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit/